In [45]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN


In [46]:
fp = Path('Resources/Updated/play_prob1.csv')

df_2 = pd.read_csv(fp)

df_2 = pd.DataFrame(df_2)

df_2

,Unnamed: 0,posteam,play_type,down,ydstogo,qtr,week,yardline_100,drive_inside20,yards_gained,success,series_result,fg_prob,td_prob,no_score_prob,safety_prob,opp_td_prob,wp
0,0,TEN,run,1.0,10,1,1,75,0.0,-3,0,Punt,0.212601,0.406750,0.004661,0.003828,0.226051,0.546262
1,1,TEN,pass,2.0,13,1,1,78,0.0,3,1,Punt,0.167558,0.321729,0.004958,0.003786,0.296200,0.520599
2,2,TEN,pass,3.0,10,1,1,75,0.0,0,0,Punt,0.178285,0.319862,0.005425,0.003580,0.299509,0.498319
3,3,ARI,pass,1.0,10,1,1,61,1.0,38,1,First down,0.257016,0.404707,0.005188,0.004175,0.213975,0.522434
4,4,ARI,run,1.0,10,1,1,23,1.0,2,0,First down,0.391608,0.529298,0.002235,0.000826,0.047409,0.616783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35761,35761,CIN,pass,1.0,10,4,22,75,0.0,17,1,First down,0.233165,0.161310,0.451072,0.001453,0.058733,0.312859
35762,35762,CIN,pass,1.0,10,4,22,58,0.0,9,1,Turnover on downs,0.268742,0.209430,0.418316,0.001627,0.046851,0.365383
35763,35763,CIN,pass,2.0,1,4,22,49,0.0,0,0,Turnover on downs,0.312642,0.235814,0.383152,0.000715,0.030652,0.406260
35764,35764,CIN,run,3.0,1,4,22,49,0.0,0,0,Turnover on downs,0.287262,0.189174,0.445391,0.000627,0.035990,0.331002


In [47]:
df_2['play_type'] = df_2['play_type'].astype(str).str.replace('run' , '1')

df_2['play_type'] = df_2['play_type'].astype(str).str.replace('pass' , '0')

df_2['play_type'] = df_2['play_type'].astype('float')

df_2


,Unnamed: 0,posteam,play_type,down,ydstogo,qtr,week,yardline_100,drive_inside20,yards_gained,success,series_result,fg_prob,td_prob,no_score_prob,safety_prob,opp_td_prob,wp
0,0,TEN,1.0,1.0,10,1,1,75,0.0,-3,0,Punt,0.212601,0.406750,0.004661,0.003828,0.226051,0.546262
1,1,TEN,0.0,2.0,13,1,1,78,0.0,3,1,Punt,0.167558,0.321729,0.004958,0.003786,0.296200,0.520599
2,2,TEN,0.0,3.0,10,1,1,75,0.0,0,0,Punt,0.178285,0.319862,0.005425,0.003580,0.299509,0.498319
3,3,ARI,0.0,1.0,10,1,1,61,1.0,38,1,First down,0.257016,0.404707,0.005188,0.004175,0.213975,0.522434
4,4,ARI,1.0,1.0,10,1,1,23,1.0,2,0,First down,0.391608,0.529298,0.002235,0.000826,0.047409,0.616783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35761,35761,CIN,0.0,1.0,10,4,22,75,0.0,17,1,First down,0.233165,0.161310,0.451072,0.001453,0.058733,0.312859
35762,35762,CIN,0.0,1.0,10,4,22,58,0.0,9,1,Turnover on downs,0.268742,0.209430,0.418316,0.001627,0.046851,0.365383
35763,35763,CIN,0.0,2.0,1,4,22,49,0.0,0,0,Turnover on downs,0.312642,0.235814,0.383152,0.000715,0.030652,0.406260
35764,35764,CIN,1.0,3.0,1,4,22,49,0.0,0,0,Turnover on downs,0.287262,0.189174,0.445391,0.000627,0.035990,0.331002


In [4]:
print(df_2.columns.tolist())


['Unnamed: 0', 'posteam', 'play_type', 'down', 'ydstogo', 'qtr', 'week', 'yardline_100', 'drive_inside20', 'yards_gained', 'success', 'series_result', 'fg_prob', 'td_prob', 'no_score_prob', 'safety_prob', 'opp_td_prob', 'wp']


In [5]:
df_2 = df_2[['success' , 'yards_gained' , 'down' , 'ydstogo' , 'play_type']].convert_dtypes()

In [6]:
df_2 = df_2[df_2.down != 0 ]

In [7]:
df_2['down'].isnull().sum()

0

In [48]:
df_2 = df_2.dropna(axis = 'columns' , how='any')

df_2 = df_2.dropna()

df_2


,Unnamed: 0,posteam,play_type,ydstogo,qtr,week,yardline_100,yards_gained,success,series_result,fg_prob,td_prob,no_score_prob,safety_prob,opp_td_prob,wp
0,0,TEN,1.0,10,1,1,75,-3,0,Punt,0.212601,0.406750,0.004661,0.003828,0.226051,0.546262
1,1,TEN,0.0,13,1,1,78,3,1,Punt,0.167558,0.321729,0.004958,0.003786,0.296200,0.520599
2,2,TEN,0.0,10,1,1,75,0,0,Punt,0.178285,0.319862,0.005425,0.003580,0.299509,0.498319
3,3,ARI,0.0,10,1,1,61,38,1,First down,0.257016,0.404707,0.005188,0.004175,0.213975,0.522434
4,4,ARI,1.0,10,1,1,23,2,0,First down,0.391608,0.529298,0.002235,0.000826,0.047409,0.616783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35761,35761,CIN,0.0,10,4,22,75,17,1,First down,0.233165,0.161310,0.451072,0.001453,0.058733,0.312859
35762,35762,CIN,0.0,10,4,22,58,9,1,Turnover on downs,0.268742,0.209430,0.418316,0.001627,0.046851,0.365383
35763,35763,CIN,0.0,1,4,22,49,0,0,Turnover on downs,0.312642,0.235814,0.383152,0.000715,0.030652,0.406260
35764,35764,CIN,1.0,1,4,22,49,0,0,Turnover on downs,0.287262,0.189174,0.445391,0.000627,0.035990,0.331002


In [49]:
x_2 = {'down_1': '1' , 'down_2': '2' , 'down_3': '3' , 'down_4': '4'}

df_2 = df_2.replace(x_2)

df_2.reset_index(inplace=True, drop=True)

df_2.head(10)

,Unnamed: 0,posteam,play_type,ydstogo,qtr,week,yardline_100,yards_gained,success,series_result,fg_prob,td_prob,no_score_prob,safety_prob,opp_td_prob,wp
0,0,TEN,1.0,10,1,1,75,-3,0,Punt,0.212601,0.406750,0.004661,0.003828,0.226051,0.546262
1,1,TEN,0.0,13,1,1,78,3,1,Punt,0.167558,0.321729,0.004958,0.003786,0.296200,0.520599
2,2,TEN,0.0,10,1,1,75,0,0,Punt,0.178285,0.319862,0.005425,0.003580,0.299509,0.498319
3,3,ARI,0.0,10,1,1,61,38,1,First down,0.257016,0.404707,0.005188,0.004175,0.213975,0.522434
4,4,ARI,1.0,10,1,1,23,2,0,First down,0.391608,0.529298,0.002235,0.000826,0.047409,0.616783
5,5,ARI,0.0,18,1,1,31,1,0,First down,0.495410,0.376114,0.002672,0.000989,0.073216,0.591982
6,6,ARI,0.0,17,1,1,30,21,1,First down,0.560816,0.290844,0.003365,0.001014,0.086453,0.580620
7,7,ARI,1.0,9,1,1,9,0,0,Field goal,0.332625,0.622383,0.001482,0.000807,0.025766,0.630427
8,8,ARI,0.0,24,1,1,24,8,0,Field goal,0.655296,0.245268,0.005205,0.001212,0.057383,0.585754
9,9,ARI,0.0,16,1,1,16,0,0,Field goal,0.741480,0.170239,0.004201,0.000994,0.051482,0.566266


In [50]:
X_2 = df_2.drop('play_type' , axis=1)

X_2 = pd.get_dummies(X_2)

# Create our target
y_2 = df_2['play_type'].astype('int')

In [51]:
X_2.describe()

,Unnamed: 0,ydstogo,qtr,week,yardline_100,yards_gained,success,fg_prob,td_prob,no_score_prob,...,series_result_Field goal,series_result_First down,series_result_Missed field goal,series_result_Opp touchdown,series_result_Punt,series_result_QB kneel,series_result_Safety,series_result_Touchdown,series_result_Turnover,series_result_Turnover on downs
count,35766.0000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,...,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000,35766.000000
mean,17882.5000,8.402561,2.569843,9.946513,50.203433,5.495806,0.442347,0.259290,0.378873,0.127620,...,0.071744,0.562042,0.011995,0.004697,0.183079,0.000475,0.000475,0.083962,0.037438,0.039171
std,10324.8992,4.003877,1.135784,5.612444,24.544517,8.504883,0.496672,0.112385,0.150133,0.188828,...,0.258067,0.496143,0.108863,0.068376,0.386737,0.021797,0.021797,0.277335,0.189835,0.194005
min,0.0000,0.000000,1.000000,1.000000,1.000000,-28.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8941.2500,6.000000,2.000000,5.000000,31.000000,0.000000,0.000000,0.189282,0.299727,0.007984,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,17882.5000,10.000000,3.000000,10.000000,54.000000,3.000000,0.000000,0.233192,0.379230,0.035498,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,26823.7500,10.000000,4.000000,15.000000,71.000000,8.000000,1.000000,0.317141,0.456565,0.168733,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,35765.0000,43.000000,5.000000,22.000000,99.000000,91.000000,1.000000,0.907078,0.933002,0.977925,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
y_2.value_counts()

0    21010
1    14756
Name: play_type, dtype: int64

In [59]:
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, random_state=1)

#### BalancedRandomForestClassifier

In [60]:
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

brfc.fit(X_2_train, y_2_train)

y_2_pred = brfc.predict(X_2_test)

y_2_pred = brfc.predict(X_2_test)

In [61]:
balanced_accuracy_score(y_2_test , y_2_pred)

balanced_accuracy_score(y_2_test , y_2_pred)

confusion_matrix(y_2_test, y_2_pred)

print(classification_report_imbalanced(y_2_test, y_2_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.80      0.76      0.74      0.78      0.75      0.57      5160
          1       0.70      0.74      0.76      0.72      0.75      0.57      3782

avg / total       0.76      0.76      0.75      0.76      0.75      0.57      8942



In [62]:
results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)

results.head(10)

,Prediction,Actual
0,0,0
1,1,1
2,0,1
3,0,0
4,1,1
5,1,1
6,0,0
7,0,0
8,0,0
9,1,0


In [63]:
results_2 = pd.DataFrame({"Prediction": y_2_pred})

results_3 = pd.DataFrame({"Actual": y_2_test})

a = accuracy_score(results_2['Prediction'], results_3['Actual'])
    
print(f'accuracy: {a*100:.2f}%')

accuracy: 75.54%


In [64]:
conf_matrix = confusion_matrix(y_2_test, y_2_pred)

table = pd.DataFrame(conf_matrix , index = ['Pass' , 'Run'], columns=['Actual' , 'Predicted'])

table

,Actual,Predicted
Pass,3944,1216
Run,971,2811


#### EasyEnsembleClassifier

In [65]:
eec = EasyEnsembleClassifier(n_estimators=100 , random_state=1)

eec = eec.fit(X_2_train , y_2_train)

y_2_pred = eec.predict(X_2_test)

In [66]:
balanced_accuracy_score(y_2_test , y_2_pred)

confusion_matrix(y_2_test , y_2_pred)

print(classification_report_imbalanced(y_2_test , y_2_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.81      0.75      0.76      0.78      0.76      0.57      5160
          1       0.69      0.76      0.75      0.73      0.76      0.57      3782

avg / total       0.76      0.76      0.76      0.76      0.76      0.57      8942



In [67]:
ecc_results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)

ecc_results.head(10)

,Prediction,Actual
0,0,0
1,1,1
2,0,1
3,0,0
4,1,1
5,1,1
6,0,0
7,0,0
8,0,0
9,1,0


In [70]:
ecc_results_1 = pd.DataFrame({"Prediction": y_2_pred})

ecc_results_2 = pd.DataFrame({"Actual": y_2_test})

a_2 = accuracy_score(ecc_results_1['Prediction'], ecc_results_2['Actual'])
    
print(f'accuracy: {a_2*100:.2f}%')

accuracy: 75.53%


In [71]:
conf_matrix_2 = confusion_matrix(y_2_test, y_2_pred)

table_2 = pd.DataFrame(conf_matrix_2 , index = ['Pass' , 'Run'], columns=['Actual' , 'Predicted'])

table_2

,Actual,Predicted
Pass,3861,1299
Run,889,2893


#### RandomOverSampler

In [72]:
ros = RandomOverSampler(random_state=1)

X_2_resamp, y_2_resamp = ros.fit_resample(X_2_train, y_2_train)

Counter(y_2_resamp)

Counter({1: 15850, 0: 15850})

In [73]:
model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_2_resamp, y_2_resamp)

LogisticRegression(random_state=1)

In [85]:
y_2_pred = model.predict(X_2_test)

balanced_accuracy_score(y_2_test, y_2_pred)

print(classification_report_imbalanced(y_2_test , y_2_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.66      0.30      0.79      0.41      0.49      0.22      5160
          1       0.45      0.79      0.30      0.57      0.49      0.25      3782

avg / total       0.57      0.51      0.58      0.48      0.49      0.23      8942



In [88]:
ros_results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)

ros_results.head(10)

,Prediction,Actual
0,1.0,0
1,0.0,1
2,0.0,1
3,0.0,0
4,1.0,1
5,1.0,1
6,1.0,0
7,1.0,0
8,1.0,0
9,1.0,0


In [87]:
ros_results_1 = pd.DataFrame({"Prediction": y_2_pred})

ros_results_2 = pd.DataFrame({"Actual": y_2_test})

a_3 = accuracy_score(ros_results_1['Prediction'], ros_results_2['Actual'])
    
print(f'accuracy: {a_3*100:.2f}%')

accuracy: 50.62%


In [67]:
ros_matrix = confusion_matrix(y_2_test, y_2_pred)

ros_df = pd.DataFrame(
    ros_matrix, 
    index = ['Pass' , 'Run'] , 
    columns=['Actual' , 'Predicted'])

ros_df

,Actual,Predicted
Pass,2890,2343
Run,950,2719


#### Smote Oversampling 


In [74]:
X_2_train = X_2_train.astype('float') 

y_2_train = y_2_train.astype('float')

X_2_resamp, y_2_resamp = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(X_2_train, y_2_train)


In [75]:
model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_2_resamp, y_2_resamp)

LogisticRegression(random_state=1)

In [111]:
y_2_pred_s = model.predict(X_2_test)

balanced_accuracy_score(y_2_test, y_2_pred_s)

0.6477030295558664

In [107]:
print(classification_report_imbalanced(y_2_test, y_2_pred_s))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.55      0.74      0.64      0.64      0.40      5233
          1       0.54      0.74      0.55      0.62      0.64      0.42      3669

avg / total       0.66      0.63      0.66      0.63      0.64      0.41      8902



In [ ]:
smot_results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)

smot_results.head(10)

In [108]:
smot_results_1 = pd.DataFrame({"Prediction": y_2_pred_s})

smot_results_2 = pd.DataFrame({"Actual": y_2_test})

a_4 = accuracy_score(smot_results_1['Prediction'], smot_results_2['Actual'])
    
print(f'accuracy: {a_4*100:.2f}%')

,Prediction
0,0.00%
1,100.00%
2,100.00%
3,0.00%
4,100.00%


In [86]:
matrix_sm = confusion_matrix(y_2_test, y_2_pred_s)

smot_results = pd.DataFrame(matrix_sm, index = ['Pass' , 'Run'], columns=['Actual' , 'Predicted'])

smot_results

,Actual,Predicted
Pass,2886,2347
Run,941,2728



#### Undersampling


In [97]:
X_2_train = X_2_train.astype('int') 

y_2_train = y_2_train.astype('int')

In [88]:
cc = ClusterCentroids(random_state=1)

X_2_resample_3, y_2_resample_3 = cc.fit_resample(X_2_train, y_2_train)

Counter(y_2_resample_3)


Counter({0: 11041, 1: 11041})

In [89]:
model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_2_resample_3, y_2_resample_3)

y_2_pred_cc = model.predict(X_2_test)

In [119]:
balanced_accuracy_score(y_2_test, y_2_pred_cc)

0.6477030295558664

In [126]:
print(classification_report_imbalanced(y_2_test, y_2_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.55      0.75      0.64      0.64      0.40      5233
          1       0.54      0.75      0.55      0.62      0.64      0.42      3669

avg / total       0.67      0.63      0.66      0.63      0.64      0.41      8902



In [ ]:
cc_results = pd.DataFrame({"Prediction": y_2_pred_cc, "Actual": y_2_test}).reset_index(drop=True)

cc_results.head(10)

In [116]:
cc_results_1 = pd.DataFrame({"Prediction": y_2_pred_cc})

cc_results_2 = pd.DataFrame({"Actual": y_2_test})

a_5 = accuracy_score(cc_results_1['Prediction'], cc_results_2['Actual'])
    
print(f'accuracy: {a_5*100:.2f}%')

,Prediction
0,0
1,1
2,1
3,0
4,1


In [125]:
matrix_cc = confusion_matrix(y_2_test, y_2_pred_cc)

cc_results = pd.DataFrame(
    matrix_cc, 
    index = ['Pass' , 'Run'], 
    columns=['Actual' , 'Predicted'])

cc_results

,Actual,Predicted
Pass,2878,2355
Run,934,2735


#### Under and over sampling 

In [74]:
y_2 = y_2.astype('float')

X_2 = X_2.astype('float')

In [75]:
smote = SMOTEENN(random_state=0)

X_resamp_teen, y_resamp_teen = smote.fit_resample(X_2, y_2)

Counter(y_resamp_teen)

Counter({0.0: 4067, 1.0: 6792})

In [76]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_resamp_teen, y_resamp_teen)

LogisticRegression(random_state=1)

In [77]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_2_test, y_2_pred)

0.7565974997847822

In [78]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_2_test, y_2_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.81      0.75      0.76      0.78      0.76      0.57      5160
          1       0.69      0.76      0.75      0.73      0.76      0.57      3782

avg / total       0.76      0.76      0.76      0.76      0.76      0.57      8942



In [79]:
# Display the confusion matrix
y_pred = model.predict(X_2_test)

confusion_matrix(y_2_test, y_2_pred)

array([[3861, 1299],
       [ 889, 2893]], dtype=int64)

In [80]:
steen_results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)

steen_results.head(10)

,Prediction,Actual
0,0,0
1,1,1
2,0,1
3,0,0
4,1,1
5,1,1
6,0,0
7,0,0
8,0,0
9,1,0


In [82]:
steen_results_1 = pd.DataFrame({"Prediction": y_2_pred})

steen_results_2 = pd.DataFrame({"Actual": y_2_test})

a_6 = accuracy_score(steen_results_1['Prediction'], steen_results_2['Actual'])
    
print(f'accuracy: {a_6*100:.2f}%')

accuracy: 75.53%


In [84]:
matrix_st = confusion_matrix(y_2_test, y_2_pred)

steen_results = pd.DataFrame(
    matrix_st, 
    index = ['Pass' , 'Run'], 
    columns=['Actual' , 'Predicted'])

steen_results

,Actual,Predicted
Pass,3861,1299
Run,889,2893
